## Check any incorrect ssdm id/password from config.json 

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
import json
from syncSSDM import AppDirectory

current_dir = AppDirectory.currentPath
config_dir = AppDirectory.configPath

# set options for chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-gpu")

# Read data from the JSON file
with open(config_dir, 'r') as jsonfile:
    config = json.load(jsonfile)

school = config['JOHOR_DETAIL']

driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)
driver.set_script_timeout(10)

for row in school:
    driver.get("https://ssdm.moe.gov.my/")
    print( "CHECKING FOR " + row['NAME'])
    
    login_box, pass_box = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR , "input[name='id'], input[name='password']")))
    login_box.send_keys(row['ID'])
    pass_box.send_keys(row['PASS'])
    wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@class='button']"))).click()
    
    try:
        alert = Alert(driver)
        alert.accept()
        print("is not okay")
    except Exception as e:
        print("is okay")
        driver.find_element(By.XPATH, "//a[@href='logout1.cfm']").click()
        pass  
    
        
time.sleep(1)
driver.close()

CHECKING FOR KV (ERT) AZIZAH
is not okay
CHECKING FOR KV PERDAGANGAN
is okay
CHECKING FOR KV TANJUNG PUTERI
is okay
CHECKING FOR MAKTAB SULTAN ABU BAKAR
is okay
CHECKING FOR SEKOLAH TUN FATIMAH
is not okay
CHECKING FOR SJK (CINA) CHEAH FAH
is okay
CHECKING FOR SJK (CINA) FOON YEW 1
is okay
CHECKING FOR SJK (CINA) FOON YEW 2
is okay
CHECKING FOR SJK (CINA) FOON YEW 3
is okay
CHECKING FOR SJK (CINA) FOON YEW 4
is okay
CHECKING FOR SJK (CINA) KEMPAS BARU
is okay
CHECKING FOR SJK (CINA) KUO KUANG 2
is okay
CHECKING FOR SJK (CINA) KUO KUANG
is okay
CHECKING FOR SJK (CINA) MING CHIH
is okay
CHECKING FOR SJK (CINA) MING TERK
is okay
CHECKING FOR SJK (CINA) PAI TZE
is okay
CHECKING FOR SJK (CINA) PEI HWA
is okay
CHECKING FOR SJK (CINA) PING MING


KeyboardInterrupt: 

## Test name searching algorithm for teacher name

In [ ]:
import textdistance

dropdownName = ["AHMAD SHAHRILLAFIZAN BIN HUSSAIN",
                "ABDUL HARIZ BIN ABDULL SAMAT",
                "AMIRO BIN KAMIRON",
                "ANIZAH BINTI MOHD DAUD",
                "AZIMY BIN MOHD BUKHARI",
                "AZIZUL ARIF BIN ZULKAFLEE",
                "BAHARUDDIN B ABDULLAH",
                "DZULKIFLI B BAKAR",
                "FAAIZUN BINTI  BASO SULTAN BAKRI",
                "FAEZA BINTI OMAR",
                "FARHAH BINTI JAFFAR",
                "FARISSHA BINTI ALLUWI",
                "HALIZA ABD HAMID",
                "HARTINI BINTI BUANG",
                "HASFARINA BINTI ABAS",
                "HERLINA BINTI MOHD HARON",
                "HUDA BINTI HUSSEIN",
                "JESLINDA A/P SELVARAJU",
                "JOHARI BIN JAMIL",
                "JUHAIDA HUSNA BINTI MD. SHUKOR",
                "JUWIESA ANAK BAYANG",
                "KARTINI BINTI HANAPI",
                "KHAIRIL AZHAR BIN ROSNI",
                "KHAIRUNNISA BINTI KAMARULZAMAN",
                "KHAIRUNNISYA BINTI UBAIDULLAH",
                "LIEW CHOI LING",
                "LILY SUHANY BT MAHMOOD",
                "LIM KA BOOI"
]

testName = [None,
            'LIM KA BOOI',
           'faaizun'
           ]



for i in range(len(testName)):
    if testName[i] == None:
        result = testName[0]
        print(str(i) + ". Tested name does not exist, so the result defaul to : " + str(result))
    else:
        best_match_similarity = 0
        best_match_option = None
        # Loop through the options and find the best match for the partial name
        for option in dropdownName:
            similarity = textdistance.jaro_winkler.normalized_similarity(testName[i].lower(), option.lower())
            if similarity > best_match_similarity:
                best_match_similarity = similarity
                best_match_option = option

        # Check if a best match was found and select it
        if best_match_option is not None:
            result = best_match_option
            print(str(i) + ". Tested name: ")
            print(testName[i])
            print("Predicted name:")
            print(result)
            print( "Similarity percentage: " + str(round((similarity*100),1)) + "%")
            print('')
        else:
            # Handle the case when no match was found
            result = 'none'
            
            print(result)

## Convert list of school from csv to seperate json

In [1]:
import csv
import json

# Define the input and output file paths
csv_file = '/Users/piixel/Downloads/SSDM LOGIN TERENGGANU.xlsx - Sheet1.csv'
json_file = '/Users/piixel/Desktop/data.json'

# Initialize an empty list to store the data
data = []

# Read the CSV file and convert it to a list of dictionaries
with open(csv_file, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        data.append(row)

# Write the data to a JSON file
with open(json_file, 'w') as json_file:
    json.dump(data, json_file, indent=4)

print(f'CSV file "{csv_file}" has been converted to JSON file "{json_file}".')

ImportError: cannot import name 'MainVariable' from 'wrapper_script' (/Users/piixel/Desktop/ssdmV2/wrapper_script.py)

## Mass set inactive status for single school

In [2]:
import json
import pymongo
from bson import ObjectId
from datetime import datetime
from pathlib import Path

#datetime(year, month, day)
start_date = datetime(2023, 1, 1)   #adjust accordingly
end_date = datetime(2023, 1, 1)

current_dir = Path.cwd()
config_dir = current_dir / "config.json"

# Read data from the JSON file
with open(config_dir, 'r') as jsonfile:
    config = json.load(jsonfile)

#make sure to initiate vpn to production db first
db = config['DATABASE']
DBclient = pymongo.MongoClient(db['connection'])
DBcol = DBclient[db['collection']]
DBdoc = DBcol[db['document']]

#load school detail from config
school = config['TERENGGANU_DETAIL']

#set the corresponding schoolID
filter_criteria = {"schoolID": ObjectId('64c54de0b4d7f977154f2564'),
                   "created_at": {"$gte": start_date, "$lte": end_date}
                   }
#set all filtered item to inactive (1:active, 2:inactive)
results = DBdoc.update_many(filter_criteria, {"$set": {"status": 2}})

## Check number of submissions for a time period

In [16]:
import json
import pymongo
from bson import ObjectId
from datetime import datetime
from pathlib import Path

#datetime(year, month, day)
start_date = datetime(2023, 10, 1)   #adjust accordingly
end_date = datetime(2023, 10, 31)

current_dir = Path.cwd()
config_dir = current_dir / "config.json"

# Read data from the JSON file
with open(config_dir, 'r') as jsonfile:
    config = json.load(jsonfile)

#make sure to initiate vpn to production db first
db = config['DATABASE']
DBclient = pymongo.MongoClient(db['connection'])
DBcol = DBclient[db['collection']]
DBdoc = DBcol[db['document']]

#load school detail from config
school = config['JOHOR']

#set the corresponding schoolID
filter_criteria = {"schoolID": ObjectId('62bc5d3d9ab3130ba059a0e9'), #change shccol ID appropriately
                   "created_at": {"$gte": start_date, "$lte": end_date}
                   }
submissions = DBdoc.count_documents(filter_criteria)
print(f"Number of submissions: {submissions}")

Number of submissions: 22553
